In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

# Load the dataset
data = pd.read_csv(r"C:\Users\truon\Downloads\Waterlevel_Final.csv")
# Ensure the date column is a datetime object
data['Time'] = pd.to_datetime(data['Time'])
data = data.drop('Unnamed: 0',axis=1)
data['Month']=[i.month for i in data['Time']]
data['Week']=[(i.month*30+i.day)//7 for i in data['Time']]
# Sort the data by date
data = data.sort_values('Time')
time = data['Time']
data = data.drop('Time',axis= 1)
data_scale = scaler.fit_transform(data)
data = pd.DataFrame(data_scale,columns=data.columns)
data['Time'] = time
# Display the first few rows of the dataset
data.head()

,Waterlevel,Month,Week,Time
0,0.116279,0.0,0.0,2008-01-01 01:00:00
1,0.107558,0.0,0.0,2008-01-01 04:00:00
2,0.098837,0.0,0.0,2008-01-01 07:00:00
3,0.100291,0.0,0.0,2008-01-01 10:00:00
4,0.101744,0.0,0.0,2008-01-01 13:00:00


In [2]:
def create_features(data, window_size):
    
    features = pd.DataFrame()
    features['Waterlevel'] = data['Waterlevel']
    features['Month'] = data['Month']
    features['Week'] = data['Week']
    for i in range(window_size):
        features[f'lag_{i+1}'] = data['Waterlevel'].shift(i+1)
    
    return features.dropna()

# Define the window size
window_size = 5
# Create features
features = create_features(data, window_size)
features.index = data['Time'][window_size:]
data['rolling_mean'] = data['Waterlevel'].rolling(window=5).mean()
data['rolling_std'] = data['Waterlevel'].rolling(window=5).std()
features['rolling_mean'] = data["rolling_mean"].tail(len(data)-5).values
features['rolling_std'] = data["rolling_std"].tail(len(data)-5).values
# Split the dataset into train and test sets
train_size = int(len(features) * 0.8)
train, test = features[:train_size], features[train_size:]
# test.index = data['Time'][train_size + window_size:]
# train.index = data['Time'][window_size:train_size+window_size]
# Extract the feature and target variables
X_train = train.drop('Waterlevel', axis=1)
y_train = train['Waterlevel']
X_test = test.drop('Waterlevel', axis=1)
y_test = test['Waterlevel']

In [3]:
features

,Waterlevel,Month,Week,lag_1,lag_2,lag_3,lag_4,lag_5,rolling_mean,rolling_std
Time,,,,,,,,,,
2008-01-01 16:00:00,0.105136,0.0,0.0,0.101744,0.100291,0.098837,0.107558,0.116279,0.102713,0.003577
2008-01-01 19:00:00,0.108527,0.0,0.0,0.105136,0.101744,0.100291,0.098837,0.107558,0.102907,0.003915
2008-01-01 22:00:00,0.106589,0.0,0.0,0.108527,0.105136,0.101744,0.100291,0.098837,0.104457,0.003402
2008-01-02 01:00:00,0.104651,0.0,0.0,0.106589,0.108527,0.105136,0.101744,0.100291,0.105329,0.002508
2008-01-02 04:00:00,0.103682,0.0,0.0,0.104651,0.106589,0.108527,0.105136,0.101744,0.105717,0.001889
...,...,...,...,...,...,...,...,...,...,...
2017-12-31 10:00:00,0.129845,1.0,1.0,0.135659,0.117248,0.098837,0.109496,0.120155,0.118217,0.014933
2017-12-31 13:00:00,0.124031,1.0,1.0,0.129845,0.135659,0.117248,0.098837,0.109496,0.121124,0.014208
2017-12-31 16:00:00,0.113372,1.0,1.0,0.124031,0.129845,0.135659,0.117248,0.098837,0.124031,0.009064


In [4]:
test

,Waterlevel,Month,Week,lag_1,lag_2,lag_3,lag_4,lag_5,rolling_mean,rolling_std
Time,,,,,,,,,,
2016-01-01 13:00:00,0.110465,0.0,0.0,0.085271,0.060078,0.072674,0.085271,0.095930,0.082752,0.018684
2016-01-01 16:00:00,0.099806,0.0,0.0,0.110465,0.085271,0.060078,0.072674,0.085271,0.085659,0.020240
2016-01-01 19:00:00,0.089147,0.0,0.0,0.099806,0.110465,0.085271,0.060078,0.072674,0.088953,0.018894
2016-01-01 22:00:00,0.078488,0.0,0.0,0.089147,0.099806,0.110465,0.085271,0.060078,0.092636,0.012608
2016-01-02 01:00:00,0.067829,0.0,0.0,0.078488,0.089147,0.099806,0.110465,0.085271,0.089147,0.016853
...,...,...,...,...,...,...,...,...,...,...
2017-12-31 10:00:00,0.129845,1.0,1.0,0.135659,0.117248,0.098837,0.109496,0.120155,0.118217,0.014933
2017-12-31 13:00:00,0.124031,1.0,1.0,0.129845,0.135659,0.117248,0.098837,0.109496,0.121124,0.014208
2017-12-31 16:00:00,0.113372,1.0,1.0,0.124031,0.129845,0.135659,0.117248,0.098837,0.124031,0.009064


In [5]:
train

,Waterlevel,Month,Week,lag_1,lag_2,lag_3,lag_4,lag_5,rolling_mean,rolling_std
Time,,,,,,,,,,
2008-01-01 16:00:00,0.105136,0.0,0.0,0.101744,0.100291,0.098837,0.107558,0.116279,0.102713,0.003577
2008-01-01 19:00:00,0.108527,0.0,0.0,0.105136,0.101744,0.100291,0.098837,0.107558,0.102907,0.003915
2008-01-01 22:00:00,0.106589,0.0,0.0,0.108527,0.105136,0.101744,0.100291,0.098837,0.104457,0.003402
2008-01-02 01:00:00,0.104651,0.0,0.0,0.106589,0.108527,0.105136,0.101744,0.100291,0.105329,0.002508
2008-01-02 04:00:00,0.103682,0.0,0.0,0.104651,0.106589,0.108527,0.105136,0.101744,0.105717,0.001889
...,...,...,...,...,...,...,...,...,...,...
2015-12-31 22:00:00,0.095930,1.0,1.0,0.106589,0.117248,0.127907,0.097868,0.067829,0.109109,0.013472
2016-01-01 01:00:00,0.085271,0.0,0.0,0.095930,0.106589,0.117248,0.127907,0.097868,0.106589,0.016853
2016-01-01 04:00:00,0.072674,0.0,0.0,0.085271,0.095930,0.106589,0.117248,0.127907,0.095543,0.017477


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV


# Train the model
rf_model = RandomForestRegressor(random_state=42,n_estimators=1000,max_features=4)
rf_model.fit(X_train, y_train)


RandomForestRegressor(max_features=4, n_estimators=1000, random_state=42)

In [7]:
import numpy as np
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

print(importances)

[5.96314348e-05 1.27842729e-04 4.57702200e-01 1.64078221e-01
 7.65145494e-02 2.74099529e-02 8.43507692e-03 2.64446971e-01
 1.22555473e-03]


In [8]:
import numpy as np
from datetime import timedelta
# best_rf= RandomForestRegressor(n_estimators=500,random_state=42)
# best_rf.fit(X_train,y_train)
# Convert the start_date and end_date to datetime format
start_date = pd.to_datetime('2016-01-01 13:00:00')  # Adjust this value as needed
end_date = pd.to_datetime('2016-01-05 13:00:00')    # Adjust this value as needed
# Function to forecast for a specific time range using Random Forest
def forecast_rf(train, test, rf_model, window_size, start_date, end_date):
    test_subset = test[(test.index >= start_date) & (test.index <= end_date)]
    rf_forecast = []
    history = train.loc[[start_date - timedelta(hours=3)]]

    for i in range(len(test_subset)):
        X_history = history.drop('Waterlevel', axis=1).tail(1)
        forecast = rf_model.predict(X_history)
        rf_forecast.append(forecast[0])
        
        # Update the history with the new forecasted value
        new_row = pd.DataFrame()
        new_row.loc[0,'Waterlevel']= forecast[0]
        for i in range(window_size):
            new_row.loc[0,f'lag_{i+1}']=history.iloc[-1, i]
        rolling = new_row.values[0][:0:-1]
        rolling = pd.DataFrame({"Waterlevel":rolling})
        new_row['Month']= test_subset['Month'][i]
        new_row['Week']= test_subset['Week'][i]
        new_row.loc[0,'rolling_mean'] = rolling['Waterlevel'].rolling(window=5).mean().tail(1).values[0]
        new_row.loc[0,'rolling_std'] = rolling['Waterlevel'].rolling(window=5).std().tail(1).values[0]
        # print(rolling)
        # print(new_row)
        history = pd.concat([history, new_row], ignore_index=True)
    # print(history)
    return pd.Series(rf_forecast, index=test_subset.index),test_subset

# Forecast using Random Forest for the specified time range
rf_forecast,test_subset = forecast_rf(features, test, rf_model , window_size, start_date, end_date)


C:\Users\truon\AppData\Local\Temp\ipykernel_14852\216335368.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row['Month']= test_subset['Month'][i]
C:\Users\truon\AppData\Local\Temp\ipykernel_14852\216335368.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row['Week']= test_subset['Week'][i]
C:\Users\truon\AppData\Local\Temp\ipykernel_14852\216335368.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row['Month'

In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import timedelta

# Assuming you have defined your forecast_rf function somewhere

# Function to calculate Similarity
def similarity(y_true, y_pred):
    max_x = max(y_true)
    min_x = min(y_true)
    T = len(y_true)
    sum_sim = sum(1 / (1 + abs(y_pred[i] - y_true[i]) / (max_x - min_x)) for i in range(T))
    similarity_score = (1 / T) * sum_sim
    return similarity_score

# Function to calculate FSD
def fsd(y_true, y_pred):
    std_y = np.std(y_true)
    std_pred = np.std(y_pred)
    fsd_score = 2 * abs(std_pred - std_y) / (std_pred + std_y)
    return fsd_score

# Function to calculate NSE
def nse(y_true, y_pred):
    numerator = sum((y_true[i] - y_pred[i])**2 for i in range(len(y_true)))
    denominator = sum((y_true[i] - np.mean(y_true))**2 for i in range(len(y_true)))
    nse_score = 1 - (numerator / denominator)
    return nse_score

# Function to calculate R score (Pearson correlation coefficient)
def r_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0, 1]

# Function to forecast for different horizons and calculate metrics
def forecast_and_evaluate(train, test, rf_model, window_size, start_date, end_date):
    forecast_horizons = ['12h', '24h', '48h', '72h', '5 days']
    forecast_results = {}

    for horizon in forecast_horizons:
        if horizon == '12h':
            end_date_horizon = start_date + timedelta(hours=12)
        elif horizon == '24h':
            end_date_horizon = start_date + timedelta(hours=24)
        elif horizon == '48h':
            end_date_horizon = start_date + timedelta(hours=48)
        elif horizon == '72h':
            end_date_horizon = start_date + timedelta(hours=72)
        elif horizon == '5 days':
            end_date_horizon = start_date + timedelta(days=5)

        # Perform forecasting
        rf_forecast,test_subset = forecast_rf(train, test, rf_model, window_size, start_date, end_date_horizon)

        # Extract actual values from test set for the horizon
        actual_values = test[(test.index >= start_date) & (test.index <= end_date_horizon)]['Waterlevel']

        # Calculate metrics
        mae = mean_absolute_error(actual_values, rf_forecast)
        rmse = np.sqrt(mean_squared_error(actual_values, rf_forecast))
        r = r_score(actual_values, rf_forecast)
        sim_score = similarity(actual_values, rf_forecast)
        fsd_score = fsd(actual_values, rf_forecast)
        nse_score = nse(actual_values, rf_forecast)

        # Store results
        forecast_results[horizon] = {
            'MAE': mae,
            'RMSE': rmse,
            'R score': r,
            'Similarity': sim_score,
            'FSD': fsd_score,
            'NSE': nse_score
        }

    return forecast_results,test_subset,rf_forecast

# Example usage:
# Replace 'features', 'test', 'rf_model', 'window_size', 'start_date', and 'end_date' with your actual data and model
forecast_results,test_subset,rf_forecast = forecast_and_evaluate(features, test, rf_model, window_size, start_date, end_date)
print("Forecast Results:")
for horizon, metrics in forecast_results.items():
    print(f"For {horizon}:")
    print(f"MAE: {metrics['MAE']}")
    print(f"RMSE: {metrics['RMSE']}")
    print(f"R score: {metrics['R score']}")
    print(f"Similarity: {metrics['Similarity']}")
    print(f"FSD: {metrics['FSD']}")
    print(f"NSE: {metrics['NSE']}")
    print("---------------------------")


C:\Users\truon\AppData\Local\Temp\ipykernel_14852\216335368.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row['Month']= test_subset['Month'][i]
C:\Users\truon\AppData\Local\Temp\ipykernel_14852\216335368.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row['Week']= test_subset['Week'][i]
C:\Users\truon\AppData\Local\Temp\ipykernel_14852\216335368.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row['Month'

Forecast Results:
For 12h:
MAE: 0.014633430232558278
RMSE: 0.016293032041021106
R score: 0.08593002266283323
Similarity: 0.7546244207108966
FSD: 0.9150599989715903
NSE: -0.16828244731408182
---------------------------
For 24h:
MAE: 0.019740794573643496
RMSE: 0.022334970130050345
R score: -0.13545826962539326
Similarity: 0.7705633900902683
FSD: 1.0266589141274327
NSE: -0.19241734145886635
---------------------------
For 48h:
MAE: 0.01945192088463298
RMSE: 0.021509550850192308
R score: -0.05065892481877867
Similarity: 0.7706518252139782
FSD: 0.944225109873861
NSE: -0.17254509340006652
---------------------------
For 72h:
MAE: 0.0187277713178295
RMSE: 0.020875670660008876
R score: -0.025714212093598286
Similarity: 0.7779308694516138
FSD: 0.8803535094324146
NSE: -0.23295251018494034
---------------------------
For 5 days:
MAE: 0.016484980620155125
RMSE: 0.019417384215669672
R score: -0.029916534021275032
Similarity: 0.8032495810521261
FSD: 0.8077449110704504
NSE: -0.22242765086629435
-----

C:\Users\truon\AppData\Local\Temp\ipykernel_14852\216335368.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row['Month']= test_subset['Month'][i]
C:\Users\truon\AppData\Local\Temp\ipykernel_14852\216335368.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row['Week']= test_subset['Week'][i]
C:\Users\truon\AppData\Local\Temp\ipykernel_14852\216335368.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row['Month'

In [10]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=test_subset.index, y=test_subset['Waterlevel'],
                         mode='lines+markers',
                         name='Waterlevel',
                         hovertemplate='Date: %{x}<br>Waterlevel: %{y}<extra></extra>'))
fig.add_trace(go.Scatter(x=test_subset.index, y=rf_forecast,
                         mode='lines+markers',
                         name='Waterlevel-Forecast',
                         hovertemplate='Date: %{x}<br>Waterlevel: %{y}<extra></extra>'))
fig.update_layout(title='Waterlevel',
                  xaxis_title='Time',
                  yaxis_title='Waterlevel',
                  hovermode='x unified')
fig.show()

In [11]:
print(X_train,y_train,features)

                     Month  Week     lag_1     lag_2     lag_3     lag_4  \
Time                                                                       
2008-01-01 16:00:00    0.0   0.0  0.101744  0.100291  0.098837  0.107558   
2008-01-01 19:00:00    0.0   0.0  0.105136  0.101744  0.100291  0.098837   
2008-01-01 22:00:00    0.0   0.0  0.108527  0.105136  0.101744  0.100291   
2008-01-02 01:00:00    0.0   0.0  0.106589  0.108527  0.105136  0.101744   
2008-01-02 04:00:00    0.0   0.0  0.104651  0.106589  0.108527  0.105136   
...                    ...   ...       ...       ...       ...       ...   
2015-12-31 22:00:00    1.0   1.0  0.106589  0.117248  0.127907  0.097868   
2016-01-01 01:00:00    0.0   0.0  0.095930  0.106589  0.117248  0.127907   
2016-01-01 04:00:00    0.0   0.0  0.085271  0.095930  0.106589  0.117248   
2016-01-01 07:00:00    0.0   0.0  0.072674  0.085271  0.095930  0.106589   
2016-01-01 10:00:00    0.0   0.0  0.060078  0.072674  0.085271  0.095930   

           

In [12]:
Forecast Results:
For 12h:
MAE: 11.645500000000004
RMSE: 15.85807510544707
R score: 0.38620119501031225
Similarity: 0.8186130716604612
FSD: 0.2780109438623875
NSE: -0.03916754566115732
---------------------------
For 24h:
MAE: 19.43488888888889
RMSE: 23.272783478418162
R score: 0.12216670836994116
Similarity: 0.7862229895161835
FSD: 0.5287118216813931
NSE: -0.21561148566084798
---------------------------
For 48h:
MAE: 23.093529411764706
RMSE: 27.48247524225532
R score: 0.0144854232357405
Similarity: 0.756795714804148
FSD: 0.22992090931521061
NSE: -0.7972941730823071
---------------------------
For 72h:
MAE: 28.46264
RMSE: 35.487898053561864
R score: -0.10943905396327523
Similarity: 0.7265869574727292
FSD: 0.08417815621888028
NSE: -2.345543144004437
---------------------------
For 5 days:
MAE: 37.89370731707317
RMSE: 46.95140871766438
R score: 0.11089459403691387
Similarity: 0.6738221042015157
FSD: 0.5371719596994515
NSE: -5.710881598127261
---------------------------

SyntaxError: invalid decimal literal (3927786168.py, line 2)